In [2]:
import os
from datetime import datetime
from IPython.display import display
from ipywidgets import interact, interact_manual
import ipywidgets as widgets
from study_common import *
from study_notebook_helper import *
import sys

study_datasets = [name for name in os.listdir(STUDY_RESULTS_PATH) if name.startswith("dataset-")]
if len(study_datasets) == 0:
    print("No study datasets available!")
    sys.exit()

# @interact(dataset=[None] + study_datasets)
@interact(dataset=study_datasets)
def main(name="erik", dataset=study_datasets[0]):
    if len(name) == 0:
        print("Please choose a name / id for the study")
        return
    if dataset is None:
        print("Hey " + name + ", please chose a dataset")
        return
    print(name + " is doing " + str(dataset))
    study: STUDY_TYPE = None
    with open(STUDY_RESULTS_PATH + dataset, "rb") as data_file:
        study = pickle.load(data_file)
    random.Random(name + dataset).shuffle(study[1])  # shuffle study entries seeded by participant and study name
    r = LocalRepo(study[0])
    r.get_tree()
    votes_out_file_name = STUDY_RESULTS_PATH + "votes-" + name + "-" + dataset[len("dataset-"):]
    try:
        with open(votes_out_file_name, "rb") as votes_file:
            votes = pickle.load(votes_file)
        print("Loaded previous vote data")
    except:
        print("No previous results found, starting from blank")
        votes = [-1 for entry in study[1]]
    print("\n" * 15)
    HEADING_TEXT = "Study Time!"
    print_html(f"<h1>{HEADING_TEXT}</h1>")
    print("\n" * 5)
    
    current_study_slider = widgets.IntSlider(
        value=0, min=0, max=len(study[1]), step=1,
        description='Entry:',
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        readout_format='d',
        layout={'width': '900px'}
    )
    @interact(current_study_entry_index=current_study_slider)
    def show_entry(current_study_entry_index):
        print_html(f"""<script>[...document.getElementsByTagName("h1")].find(e => e.innerText === "{HEADING_TEXT}").scrollIntoView();</script>""")
        if current_study_entry_index >= len(study[1]):
            missing_data_indices = [i for i, val in enumerate(votes) if val < 0]
            if len(missing_data_indices) == 0:
                print_html("<h1>You did it!</h1><br><h4>Please download your study results now, and send them to Erik.</h4>")
                try:
                    from google.colab import files

                    def do_download(btn):
                        files.download(votes_out_file_name)

                    button = widgets.Button(
                        description='Download result data now',
                        button_style='success',
                        icon='download'
                    )
                    button.on_click(do_download)
                    display(button)
                except:
                    print_html("Downloading result files is not supported here, please manually download the results file from " + votes_out_file_name)
            else:
                print_html("<h1>There is still data missing:</h1>" + ", ".join(str(i) for i in missing_data_indices))
            return
        study_entry = study[1][current_study_entry_index]
        ti = study_entry[0]
        m0 = study_entry[1]
        m1 = study_entry[2]
        commits = study_entry[3]
        commit_cells: List[str] = []
        commit_empty_cell = '<td style="border: 1px solid black; background-color: #aaa; width: 50%"></td>'
        for c in commits:
            # 0 message, 1 author, 2 date, 3 hexsha, 4 [0 files, 1 additions, 2 deletions], 5 belongs to method 1, 6 belongs to method 2
            datestring = datetime.utcfromtimestamp(c[2]).strftime('%Y-%m-%d')
            commit_cells.append(f"""<tr>
                {commit_empty_cell if not c[5] else ""}
                <td{' colspan="2"' if c[5] and c[6] else ""} style="border: 1px solid black; width: {'100' if c[5] and c[6] else '50'}%;"><center>
                    {c[1]}: <a href="https://github.com/{r.name}/commit/{c[3]}" target="_blank">{c[0]}</a><br>
                    <span style="font-family: monospace">
                        {"{:,}".format(c[4][0])} changed files:
                        <span style="color:green">+{"{:,}".format(c[4][1])}</span>
                        <span style="color:red">-{"{:,}".format(c[4][2])}</span>,
                        created on {datestring}
                    </span>
                </center></td>
                {commit_empty_cell if not c[6] else ""}
                </tr>""")
        # commit_cells.append("<tr>" + commit_empty_cell + commit_empty_cell + "</tr>")

        print_html(f"""{PRISM_HIGHLIGHTING}{OWN_STYLE}
            You will be shown multiple methods from the <a href="{r.url()}" target="_blank" style="color: blue">{study[0]}</a> repository.
            Take a close look at the two presented snippets of code.<br>
            Consider everything, not only the statements, but also variable naming, comments,
            location of these methods within the project, editing history, and relations with other code.<br>
            Click on the links to view the presented code snippets on GitHub, where you can find further context information if needed.<br>
            <br>
            Then, please select which of the given categories best describes this pair of methods.
            <br><br>
            <center>{make_path_overview_html(r, m0[0], m1[0])}</cnter>
            <br><br>
            <table border="1">
              <tr>
                <td style="font-family: monospace"><center>{path_html(r, m0[0])}</center></td>
                <td style="font-family: monospace"><center>{path_html(r, m1[0])}</center></td>
              </tr>
              <tr>
                <td><pre><code class="language-java match-braces rainbow-braces">{format_code(m0[1], m1[0])}</code></pre></td>
                <td><pre><code class="language-java match-braces rainbow-braces">{format_code(m1[1], m0[0])}</code></pre></td>
              </tr>
              <tr><td colspan="2"><center>Amount of commits: {len([c for c in commits if c[5] and not c[6]])} left only, {len([c for c in commits if not c[5] and c[6]])} right only, {len([c for c in commits if c[5] and c[6]])} shared, {len(commits)} total</center></td></tr>
              <tr><td colspan="2">
                  <div style="overflow-y: scroll; height:400px;">
                  <table style="width: 95%">{"".join(commit_cells)}</table>
                  </div>
              </td></tr>
            </table>
        """)

        print_html("<br><h2>Please chose, which category best fits to the shown method pair:</h2><br>")
        category_radio_none_option = "None of the above"
        category_radio_options = [change_text_style(name, "bold") + ": " + desc for p, name, desc in TAXONOMY] + [category_radio_none_option]
        category_radio_btn = widgets.RadioButtons(
            options=category_radio_options,
            value=(category_radio_none_option if votes[current_study_entry_index] < 0 else category_radio_options[votes[current_study_entry_index]]),
            description="Category:",
            layout={'width': 'max-content'}
        )

        # print_html(HORIZONTAL_RADIO_BUTTONS)
        print_html("<style>.widget-radio > .widget-label {width: 500px;}</style>")
        manual_save = interact_manual.options(manual_name="Save!")
        @manual_save(category=category_radio_btn)
        def vote(category):
            print("Saving...", end="")
            votes[current_study_entry_index] = -1 if category == category_radio_none_option else category_radio_options.index(category)
            with open(votes_out_file_name, "wb") as out_file:
                pickle.dump(votes, out_file)
            print("Done!")
            print("Saved data:", votes[current_study_entry_index])
            def next(btn):
                current_study_slider.value += 1

            button = widgets.Button(
                description='Next method pair',
                button_style='success',
                icon='forward'
            )
            button.on_click(next)
            display(button)
    
print("Done!")

No study datasets available!
